In [1]:
# 自动计算cell的计算时间
%load_ext autotime

%matplotlib inline
%config InlineBackend.figure_format='svg' #矢量图设置，让绘图更清晰

time: 495 ms (started: 2021-08-18 01:07:00 +08:00)


In [64]:
%%bash

# 增加更新
git add *.ipynb *.md

git remote -v

git commit -m '更新 ch13 #2  Aug 18, 2021'

git push origin master

origin	git@github.com:ustchope/handson-ml2.git (fetch)
origin	git@github.com:ustchope/handson-ml2.git (push)
[master 5763c46] 更新 ch13 #2  Aug 18, 2021
 1 file changed, 2218 insertions(+), 1 deletion(-)


To git@github.com:ustchope/handson-ml2.git
   44828bc..5763c46  master -> master


time: 4.61 s (started: 2021-08-18 03:20:11 +08:00)


In [3]:
#设置使用的gpu
import tensorflow as tf

gpus = tf.config.list_physical_devices("GPU")

if gpus:
   
    gpu0 = gpus[0] #如果有多个GPU，仅使用第0个GPU
    tf.config.experimental.set_memory_growth(gpu0, True) #设置GPU显存用量按需使用
    # 或者也可以设置GPU显存为固定使用量(例如：4G)
    #tf.config.experimental.set_virtual_device_configuration(gpu0,
    #    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=4096)]) 
    tf.config.set_visible_devices([gpu0],"GPU")

time: 3.78 s (started: 2021-08-18 01:07:29 +08:00)


# 使用 TensorFlow 加载和预处理数据

到目前为止，我们只使用了适合内存的数据集，但深度学习系统通常在不适合 RAM 的非常大的数据集上进行训练。使用其他深度学习库来实现获取大型数据集并对其进行有效预处理可能会很棘手，但由于数据 API，TensorFlow 使这件事变得容易：您只需创建一个数据集对象，并告诉它从哪里获取数据以及如何转换它. TensorFlow 负责所有实现细节，例如多线程、排队、批处理和预取。此外，数据 API 与 tf.keras 无缝协作！

现成的 Data API 可以读取文本文件（例如 CSV 文件）、具有固定大小记录的二进制文件以及使用 TensorFlow 的 TFRecord 格式的二进制文件，该格式支持不同大小的记录。 TFRecord 是一种灵活高效的二进制格式，通常包含协议缓冲区（一种开源二进制格式）。数据 API 还支持从 SQL 数据库读取数据。此外，许多开源扩展可用于从各种数据源中读取数据，例如 Google 的 BigQuery 服务。

高效读取庞大的数据集并不是唯一的困难：数据还需要进行预处理，通常是归一化。 此外，它并不总是严格由方便的数字字段组成：可能有文本特征、分类特征等。 这些需要进行编码，例如使用单热编码、词袋编码或嵌入（正如我们将看到的，嵌入是表示类别或标记的可训练密集向量）。 处理所有这些预处理的一种选择是编写您自己的自定义预处理层。 另一种是使用 Keras 提供的标准预处理层。

在本章中，我们将介绍数据 API、TFRecord 格式，以及如何创建自定义预处理层和使用标准 Keras 层。 我们还将快速浏览一下 TensorFlow 生态系统中的一些相关项目：
* TF 变换 (tf.Transform)  
使编写一个预处理函数成为可能，该函数可以在您的完整训练集上以批处理模式运行，在训练之前（以加快速度），然后导出到 TF 函数并合并到您的训练模型中，以便一旦部署 在生产中，它可以即时处理新实例。
* TF 数据集 (TFDS)  
提供了一个方便的功能来下载各种常见的数据集，包括像 ImageNet 这样的大型数据集，以及使用 Data API 操作它们的方便的数据集对象。

让我们开始吧！

## Data API

整个数据 API 围绕数据集的概念展开：正如您可能怀疑的那样，这表示数据项的序列。 通常你会使用从磁盘逐渐读取数据的数据集，但为了简单起见，让我们使用 tf.data.Dataset.from_tensor_slices() 创建一个完全在 RAM 中的数据集：

In [26]:
X = tf.range(10) # any data tensor

time: 1.47 ms (started: 2021-08-18 01:34:25 +08:00)


In [27]:
dataset = tf.data.Dataset.from_tensor_slices(X)
dataset

<TensorSliceDataset shapes: (), types: tf.int32>

time: 3.91 ms (started: 2021-08-18 01:34:26 +08:00)


from_tensor_slices() 函数接受一个张量并创建一个 tf.data.Dataset，其元素是 X 的所有切片（沿第一维），因此该数据集包含 10 个项目：张量 0、1、2、...、9。在 在这种情况下，如果我们使用 tf.data.Dataset.range(10)，我们将获得相同的数据集。

您可以简单地迭代数据集的项目，如下所示：

In [28]:
for item in dataset:
    print(item)

tf.Tensor(0, shape=(), dtype=int32)
tf.Tensor(1, shape=(), dtype=int32)
tf.Tensor(2, shape=(), dtype=int32)
tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor(4, shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)
tf.Tensor(7, shape=(), dtype=int32)
tf.Tensor(8, shape=(), dtype=int32)
tf.Tensor(9, shape=(), dtype=int32)
time: 11.1 ms (started: 2021-08-18 01:34:29 +08:00)


### 链式转换

拥有数据集后，您可以通过调用其转换方法对其应用各种转换。 每个方法都返回一个新数据集，因此您可以像这样链接转换（此链如图 13-1 所示）：

In [29]:
dataset = dataset.repeat(3).batch(7)
for item in dataset:
    print(item)

tf.Tensor([0 1 2 3 4 5 6], shape=(7,), dtype=int32)
tf.Tensor([7 8 9 0 1 2 3], shape=(7,), dtype=int32)
tf.Tensor([4 5 6 7 8 9 0], shape=(7,), dtype=int32)
tf.Tensor([1 2 3 4 5 6 7], shape=(7,), dtype=int32)
tf.Tensor([8 9], shape=(2,), dtype=int32)
time: 12.9 ms (started: 2021-08-18 01:34:35 +08:00)


![](https://tva1.sinaimg.cn/large/008i3skNgy1gtkasai5eij61dw0lstbi02.jpg)

在这个例子中，我们首先对原始数据集调用repeat()方法，它返回一个新的数据集，该数据集将原始数据集的项目重复3次。 当然，这不会将内存中的所有数据复制三遍！ （如果不带参数调用此方法，新数据集将永远重复源数据集，因此迭代数据集的代码必须决定何时停止。）然后我们在这个新数据集上调用 batch() 方法， 这再次创建了一个新的数据集。 这将把前一个数据集的项目分成七个项目的批次。 最后，我们迭代这个最终数据集的项目。 如您所见，batch() 方法必须输出大小为 2 而不是 7 的最后一批，但是如果您希望它删除最后一批以便所有批次具有完全相同的大小，则可以使用 drop_remainder=True 调用它 .

> 数据集方法不会修改数据集，而是创建新数据集，因此请确保保留对这些新数据集的引用（例如，使用 dataset = ...），否则什么都不会发生。

您还可以通过调用 map() 方法来转换项目。 例如，这将创建一个所有项目都翻倍的新数据集：

In [35]:
dataset = tf.data.Dataset.range(10)
dataset = dataset.map(lambda x: x * 2) # Items: [0,2,4,6,8,10,12]
dataset

<MapDataset shapes: (), types: tf.int64>

time: 11.4 ms (started: 2021-08-18 01:36:02 +08:00)


In [38]:
for item in dataset.take(3):
    print(dataset)

<FilterDataset shapes: (), types: tf.int64>
<FilterDataset shapes: (), types: tf.int64>
<FilterDataset shapes: (), types: tf.int64>
time: 26.5 ms (started: 2021-08-18 01:37:44 +08:00)


您将调用此函数将您想要的任何预处理应用于您的数据。 有时这将包括可能非常密集的计算，例如重塑或旋转图像，因此您通常希望生成多个线程以加快速度：就像设置 num_parallel_calls 参数一样简单。 请注意，您传递给 map() 方法的函数必须可转换为 TF 函数（请参阅第 12 章）。

map() 方法将转换应用于每个项目，而 apply() 方法将转换应用于整个数据集。 例如，以下代码将 unbatch() 函数应用于数据集（该函数目前处于实验阶段，但很可能会在未来版本中移至核心 API）。 新数据集中的每个项目将是一个单整数张量，而不是一批七个整数：

In [33]:
dataset = dataset.unbatch() # Items: 0,2,4,...
dataset

<_UnbatchDataset shapes: (), types: tf.int32>

time: 11.6 ms (started: 2021-08-18 01:35:39 +08:00)


也可以使用 filter() 方法简单地过滤数据集：

In [36]:
dataset = dataset.filter(lambda x: x < 10) # Items: 0 2 4 6 8 0 2 4 6...
dataset

<FilterDataset shapes: (), types: tf.int64>

time: 28.6 ms (started: 2021-08-18 01:36:34 +08:00)


您通常只想查看数据集中的几个项目。 您可以使用 take() 方法：

In [39]:
for item in dataset.take(3):
    print(item)

tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(2, shape=(), dtype=int64)
tf.Tensor(4, shape=(), dtype=int64)
time: 25.4 ms (started: 2021-08-18 01:37:55 +08:00)


### 打乱数据

如您所知，当训练集中的实例独立且分布相同时，梯度下降效果最好（参见第 4 章）。确保这一点的一种简单方法是使用 shuffle() 方法对实例进行混洗。它将创建一个新数据集，该数据集首先用源数据集的第一项填充缓冲区。然后，每当它被要求提供一个项目时，它会从缓冲区中随机取出一个项目，并用源数据集中的新项目替换它，直到它完全遍历源数据集。此时它会继续从缓冲区中随机取出项目，直到它为空。您必须指定缓冲区大小，并且使其足够大很重要，否则改组将不会很有效。只是不要超过您拥有的 RAM 量，即使您有足够的 RAM，也没有必要超出数据集的大小。如果您每次运行程序时都想要相同的随机顺序，您可以提供一个随机种子。例如，以下代码创建并显示一个包含 0 到 9 整数的数据集，重复 3 次，使用大小为 5 的缓冲区和 42 的随机种子进行混洗，并以批量大小为 7 进行批处理：

In [40]:
dataset = tf.data.Dataset.range(10).repeat(3) # 0 to 9, three times

time: 2.39 ms (started: 2021-08-18 01:40:09 +08:00)


In [41]:
dataset = dataset.shuffle(buffer_size=5, seed=42).batch(7)

time: 21.3 ms (started: 2021-08-18 01:40:17 +08:00)


In [42]:
for item in dataset:
    print(item)

tf.Tensor([0 2 3 6 7 9 4], shape=(7,), dtype=int64)
tf.Tensor([5 0 1 1 8 6 5], shape=(7,), dtype=int64)
tf.Tensor([4 8 7 1 2 3 0], shape=(7,), dtype=int64)
tf.Tensor([5 4 2 7 8 9 9], shape=(7,), dtype=int64)
tf.Tensor([3 6], shape=(2,), dtype=int64)
time: 12.1 ms (started: 2021-08-18 01:40:30 +08:00)


如果您在混洗数据集上调用 repeat() ，默认情况下它会在每次迭代时生成一个新订单。 这通常是一个好主意，但如果您更喜欢在每次迭代中重用相同的顺序（例如，用于测试或调试），您可以设置 reshuffle_each_iteration=False。

对于不适合内存的大型数据集，这种简单的混洗缓冲区方法可能不够用，因为缓冲区与数据集相比较小。一种解决方案是混洗源数据本身（例如，在 Linux 上，您可以使用 shuf 命令混洗文本文件）。这肯定会大大改善洗牌！即使源数据被打乱了，您通常也希望将其打乱一些，否则每个时期都会重复相同的顺序，并且模型最终可能会出现偏差（例如，由于一些偶然出现的虚假模式）源数据的顺序）。为了更多地打乱实例，一种常见的方法是将源数据拆分为多个文件，然后在训练期间以随机顺序读取它们。但是，位于同一文件中的实例最终仍会彼此靠近。为避免这种情况，您可以随机选择多个文件并同时读取它们，交错它们的记录。然后最重要的是您可以使用 shuffle() 方法添加一个混洗缓冲区。如果所有这些听起来工作量很大，请不要担心：Data API 只需几行代码就可以让这一切成为可能。让我们看看如何做到这一点。

### 来自多个文件的交错行

首先，假设您已加载加利福尼亚住房数据集，对其进行打乱（除非它已经打乱），并将其拆分为训练集、验证集和测试集。 然后将每个集合拆分为多个 CSV 文件，每个文件如下所示（每行包含八个输入特征加上目标房屋价值中值）：

In [ ]:
MedInc,HouseAge,AveRooms,AveBedrms,Popul,AveOccup,Lat,Long,MedianHouseValue
3.5214,15.0,3.0499,1.1065,1447.0,1.6059,37.63,-122.43,1.442
5.3275,5.0,6.4900,0.9910,3464.0,3.4433,33.69,-117.39,1.687
3.1,29.0,7.5423,1.5915,1328.0,2.2508,38.44,-122.98,1.621
[...]

我们还假设 train_filepaths 包含训练文件路径列表（并且您还有 valid_filepaths 和 test_filepaths）：

In [ ]:
train_filepaths
['datasets/housing/my_train_00.csv', 'datasets/housing/my_train_01.csv',...]

或者，您可以使用文件模式； 例如，train_filepaths = "datasets/housing/my_train_*.csv"。 现在让我们创建一个仅包含这些文件路径的数据集：

In [ ]:
filepath_dataset = tf.data.Dataset.list_files(train_filepaths, seed=42)

默认情况下，list_files() 函数返回一个对文件路径进行混洗的数据集。 一般来说，这是一件好事，但如果您出于某种原因不想这样做，您可以设置 shuffle=False。

接下来，您可以调用 interleave() 方法一次从五个文件中读取并交错它们的行（使用 skip() 方法跳过每个文件的第一行，即标题行）：

In [ ]:
n_readers = 5
dataset = filepath_dataset.interleave(
    lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
    cycle_length=n_readers)

interleave() 方法将创建一个数据集，该数据集将从 filepath_dataset 中提取五个文件路径，并且对于每个文件路径，它将调用您提供的函数（在本例中为 lambda）以创建一个新数据集（在本例中为 TextLineDataset） . 需要明确的是，在这个阶段总共会有七个数据集：文件路径数据集、交错数据集以及交错数据集内部创建的五个 TextLineDataset。 当我们迭代交错数据集时，它将循环遍历这五个 TextLineDatasets，一次读取一行，直到所有数据集都没有项目。 然后它将从 filepath_dataset 中获取接下来的五个文件路径并以相同的方式交错它们，依此类推，直到用完文件路径。

> 为了使交错效果最佳，最好使用相同长度的文件； 否则最长文件的结尾将不会被交错。

默认情况下， interleave() 不使用并行性； 它一次只从每个文件中读取一行，顺序。 如果您希望它实际并行读取文件，您可以将 num_parallel_calls 参数设置为您想要的线程数（注意 map() 方法也有这个参数）。 您甚至可以将其设置为 tf.data.experimental.AUTOTUNE 以使 TensorFlow 根据可用 CPU 动态选择正确数量的线程（但是，目前这是一个实验性功能）。 现在让我们看看数据集包含的内容：

In [ ]:
for line in dataset.take(5):
    print(line.numpy())

这些是随机选择的五个 CSV 文件的第一行（忽略标题行）。 看起来挺好的！ 但正如您所见，这些只是字节字符串； 我们需要解析它们并缩放数据。

### 预处理数据

让我们实现一个小函数来执行此预处理：

In [43]:
X_mean, X_std = [...] # 训练集中每个特征的均值和尺度
n_inputs = 8

In [45]:
def preprocess(line):
    defs = [0.] * n_inputs + [tf.constant([], dtype=tf.float32)]
    fields = tf.io.decode_csv(line, record_defaults=defs) x = tf.stack(fields[:-1])
    y = tf.stack(fields[-1:])
    return (x - X_mean) / X_std, y

SyntaxError: invalid syntax (<ipython-input-45-d08e14272cd4>, line 3)

time: 2.17 ms (started: 2021-08-18 02:17:14 +08:00)


让我们来看看这段代码：
* 首先，代码假设我们已经预先计算了训练集中每个特征的均值和标准差。 X_mean 和 X_std 只是包含八个浮点数的一维张量（或 NumPy 数组），每个输入特征一个。
* preprocess() 函数采用一个CSV 行并从解析它开始。为此，它使用 tf.io.decode_csv() 函数，该函数接受两个参数：第一个是要解析的行，第二个是包含 CSV 文件中每一列的默认值的数组。这个数组不仅告诉 TensorFlow 每一列的默认值，还告诉 TensorFlow 的数量和它们的类型。在这个例子中，我们告诉它所有特征列都是浮点数并且缺失值应该默认为 0，但是我们提供了一个类型为 tf.float32 的空数组作为最后一列（目标）的默认值：该数组告诉 TensorFlow该列包含浮点数，但没有默认值，因此如果遇到缺失值，它将引发异常。
* decode_csv() 函数返回标量张量列表（每列一个），但我们需要返回一维张量数组。所以我们在除最后一个（目标）之外的所有张量上调用 tf.stack() ：这会将这些张量堆叠成一维数组。然后我们对目标值执行相同的操作（这使其成为具有单个值的一维张量数组，而不是标量张量）。
* decode_csv() 函数返回标量张量列表（每列一个），但我们需要返回一维张量数组。 所以我们在除最后一个（目标）之外的所有张量上调用 tf.stack() ：这会将这些张量堆叠成一维数组。 然后我们对目标值执行相同的操作（这使其成为具有单个值的一维张量数组，而不是标量张量）。
* 最后，我们通过减去特征均值然后除以特征标准差来缩放输入特征，我们返回一个包含缩放特征和目标的元组。

让我们测试一下这个预处理函数：

In [44]:
preprocess(b'4.2083,44.0,5.3232,0.9171,846.0,2.3370,37.47,-122.2,2.782')

NameError: name 'preprocess' is not defined

time: 44.9 ms (started: 2021-08-18 02:17:07 +08:00)


看起来挺好的！ 我们现在可以将该函数应用于数据集。

### 把所有东西放在一起

为了使代码可重用，让我们将迄今为止讨论的所有内容放在一个小的辅助函数中：它将创建并返回一个数据集，该数据集将有效地从多个 CSV 文件中加载加州住房数据，对其进行预处理、洗牌、可选地重复它， 并批处理（见图 13-2）：

In [ ]:
def csv_reader_dataset(filepaths, 
                       repeat=1, 
                       n_readers=5,
                       n_read_threads=None, 
                       shuffle_buffer_size=10000,
                       n_parse_threads=5, batch_size=32):
    dataset = tf.data.Dataset.list_files(filepaths)
    dataset = dataset.interleave(
        lambda filepath: tf.data.TextLineDataset(filepath).skip(1),
        cycle_length=n_readers, num_parallel_calls=n_read_threads)
    dataset = dataset.map(preprocess, num_parallel_calls=n_parse_threads)
    dataset = dataset.shuffle(shuffle_buffer_size).repeat(repeat)
    return dataset.batch(batch_size).prefetch(1)

除了对性能很重要的最后一行 (prefetch(1)) 外，这段代码中的所有内容都应该有意义。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtkcc4m2ecj61080ncgp102.jpg)

### Prefetching

通过在最后调用 prefetch(1)，我们正在创建一个数据集，该数据集将尽最大努力始终领先一批。 换句话说，当我们的训练算法正在处理一个批次时，数据集已经在并行工作以准备好下一个批次（例如，从磁盘读取数据并对其进行预处理）。 这可以显着提高性能，如图 13-3 所示。 如果我们还确保加载和预处理是多线程的（通过在调用 interleave() 和 map() 时设置 num_parallel_calls），我们可以利用 CPU 上的多个内核，并希望比在 GPU 上运行训练步骤更短地准备一批数据 ：

通过在最后调用 prefetch(1)，我们正在创建一个数据集，该数据集将尽最大努力始终领先一批。 换句话说，当我们的训练算法正在处理一个批次时，数据集已经在并行工作以准备好下一个批次（例如，从磁盘读取数据并对其进行预处理）。 这可以显着提高性能，如图 13-3 所示。 如果我们还确保加载和预处理是多线程的（通过在调用 interleave() 和 map() 时设置 num_parallel_calls），我们可以利用 CPU 上的多个内核，并希望比在 GPU 上运行训练步骤更短地准备一批数据 ：这样 GPU 将几乎 100% 使用（除了从 CPU 到 GPU3 的数据传输时间），并且训练运行速度会快得多。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtkcfh7aroj61060ta79c02.jpg)

如果你打算购买 GPU 卡，它的处理能力和显存大小当然是非常重要的（尤其是大量 RAM 对计算机视觉至关重要）。 获得良好性能同样重要的是它的内存带宽； 这是它每秒可以进出其 RAM 的千兆字节数。

如果数据集小到可以放入内存，则可以通过使用数据集的 cache() 方法将其内容缓存到 RAM 来显着加快训练速度。 您通常应该在加载和预处理数据之后，但在混洗、重复、批处理和预取之前执行此操作。 这样，每个实例只会被读取和预处理一次（而不是每个 epoch 一次），但数据在每个 epoch 仍然会以不同的方式进行洗牌，并且仍然会提前准备下一批。

您现在知道如何构建高效的输入管道以从多个文本文件加载和预处理数据。 我们已经讨论了最常见的数据集方法，但您可能还想了解更多方法：concatenate()、zip()、window()、reduce()、shard()、flat_map() 和 padded_batch() . 还有几个类方法：from_generator() 和 from_tensors()，它们分别从 Python 生成器或张量列表创建新数据集。 请查看 API 文档以获取更多详细信息。 另请注意，在 tf.data.experimental 中有可用的实验性功能，其中许多功能可能会在未来版本中成为核心 API（例如，查看 CsvDataset 类，以及 make_csv_dataset() 方法，它负责 推断每列的类型）。

### 将数据集与 tf.keras 一起使用

现在我们可以使用 csv_reader_dataset() 函数为训练集创建数据集。 请注意，我们不需要重复它，因为这将由 tf.keras 处理。 我们还为验证集和测试集创建数据集：

In [ ]:
train_set = csv_reader_dataset(train_filepaths)
valid_set = csv_reader_dataset(valid_filepaths)
test_set = csv_reader_dataset(test_filepaths)

现在我们可以使用这些数据集简单地构建和训练 Keras 模型。 我们需要做的就是将训练和验证数据集传递给 fit() 方法，而不是 X_train、y_train、X_valid 和 y_valid：

In [ ]:
model = keras.models.Sequential([...])
model.compile([...])
model.fit(train_set, epochs=10, validation_data=valid_set)

同样，我们可以将数据集传递给evaluate() 和predict() 方法：

In [ ]:
model.evaluate(test_set)

new_set = test_set.take(3).map(lambda X, y: X) # pretend we have 3 new instances
model.predict(new_set) # a dataset containing new instances

与其他集合不同，new_set 通常不包含标签（如果包含，Keras 将忽略它们）。 请注意，在所有这些情况下，如果需要，您仍然可以使用 NumPy 数组而不是数据集（当然，它们需要先加载和预处理）。

如果您想构建自己的自定义训练循环（如第 12 章所述），您可以非常自然地遍历训练集：

In [ ]:
for X_batch, y_batch in train_set: 
    [...] # perform one Gradient Descent step

事实上，甚至可以创建一个执行整个训练循环的 TF 函数（参见第 12 章）：

In [ ]:
@tf.function
def train(model, optimizer, loss_fn, n_epochs, [...]):
    train_set = csv_reader_dataset(train_filepaths, repeat=n_epochs, [...])
    for X_batch, y_batch in train_set:
        with tf.GradientTape() as tape:
            y_pred = model(X_batch)
            main_loss = tf.reduce_mean(loss_fn(y_batch, y_pred))
            loss = tf.add_n([main_loss] + model.losses)
        grads = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

恭喜，您现在已经知道如何使用 Data API 构建强大的输入管道了！ 但是，到目前为止，我们使用的都是CSV文件，它很常见，简单方便，但效率不高，对大型或复杂的数据结构（如图像或音频）的支持不是很好。 那么让我们看看如何改用 TFRecords。

如果您对 CSV 文件（或您使用的任何其他格式）感到满意，则不必使用 TFRecords。 俗话说，没坏就别修！ 当训练期间的瓶颈是加载和解析数据时，TFRecords 很有用。

## TFRecord 格式

TFRecord 格式是 TensorFlow 的首选格式，用于存储大量数据并高效读取。 它是一种非常简单的二进制格式，只包含一系列不同大小的二进制记录（每个记录由一个长度、一个 CRC 校验和来检查长度是否被破坏，然后是实际数据，最后是一个 CRC 校验和） 数据）。 您可以使用 tf.io.TFRecordWriter 类轻松创建 TFRecord 文件：

In [ ]:
with tf.io.TFRecordWriter("my_data.tfrecord") as f: 
    f.write(b"This is the first record") 
    f.write(b"And this is the second record")

然后您可以使用 tf.data.TFRecordDataset 来读取一个或多个 TFRecord 文件：

In [ ]:
filepaths = ["my_data.tfrecord"]
dataset = tf.data.TFRecordDataset(filepaths)
for item in dataset:
    print(item)

这将输出：

In [ ]:
tf.Tensor(b'This is the first record', shape=(), dtype=string)
tf.Tensor(b'And this is the second record', shape=(), dtype=string)

> 默认情况下，TFRecordDataset 会一个一个读取文件，但您可以通过设置 num_parallel_reads 使其并行读取多个文件并交错它们的记录。 或者，您可以通过使用 list_files() 和 interleave() 获得与我们之前读取多个 CSV 文件相同的结果。

### 压缩的 TFRecord 文件

有时压缩 TFRecord 文件会很有用，尤其是当它们需要通过网络连接加载时。 您可以通过设置 options 参数来创建压缩的 TFRecord 文件：

In [ ]:
options = tf.io.TFRecordOptions(compression_type="GZIP")

with tf.io.TFRecordWriter("my_compressed.tfrecord", options) as f: 
    [...]

读取压缩的TFRecord文件时，需要指定压缩类型：

In [ ]:
dataset = tf.data.TFRecordDataset(["my_compressed.tfrecord"],
                                  compression_type="GZIP")

### Protocol Buffers 简介

尽管每个记录可以使用您想要的任何二进制格式，但 TFRecord 文件通常包含序列化的协议缓冲区（也称为 protobufs）。 这是 Google 于 2001 年开发的一种可移植、可扩展且高效的二进制格式，并于 2008 年开源； protobufs 现在被广泛使用，特别是在 gRPC，谷歌的远程过程调用系统中。 它们是使用一种简单的语言定义的，如下所示：

In [ ]:
syntax = "proto3";
message Person {
    string name = 1;
    int32 id = 2;
    repeated string email = 3; 
}

这个定义表示我们使用的是 protobuf 格式的第 3 版，它指定每个 Person 对象可以（可选）具有类型为 string 的名称、类型为 int32 的 id 和零个或多个类型为 string 的电子邮件字段。 数字 1、2 和 3 是字段标识符：它们将用于每条记录的二进制表示。 一旦在 .proto 文件中有定义，就可以编译它。 这需要 protoc，即 protobuf 编译器，在 Python（或其他一些语言）中生成访问类。 请注意，我们将使用的 protobuf 定义已经为您编译好了，它们的 Python 类是 TensorFlow 的一部分，因此您不需要使用 protoc。 您只需要知道如何在 Python 中使用 protobuf 访问类。 为了说明基础，让我们看一个简单的例子，它使用为 Person protobuf 生成的访问类（代码在注释中解释）：

## 预处理输入特征

为神经网络准备数据需要将所有特征转换为数值特征，通常将它们归一化等等。 特别是，如果您的数据包含分类特征或文本特征，则需要将它们转换为数字。 这可以在准备数据文件时使用您喜欢的任何工具（例如 NumPy、pandas 或 Scikit-Learn）提前完成。 或者，您可以在使用数据 API 加载数据时即时预处理数据（例如，使用数据集的 map() 方法，正如我们之前看到的），或者您可以直接在模型中包含一个预处理层。 现在让我们看看最后一个选项。

例如，以下是如何使用 Lambda 层实现标准化层。 对于每个特征，它减去均值并除以其标准差（加上一个微小的平滑项以避免被零除）：

In [ ]:
means = np.mean(X_train, axis=0, keepdims=True)
stds = np.std(X_train, axis=0, keepdims=True)
eps = keras.backend.epsilon()
model = keras.models.Sequential([keras.layers.Lambda(lambda inputs: (inputs - means) / (stds + eps)),
[...] # other layers
])

这不是太难！ 但是，您可能更喜欢使用一个很好的自包含自定义层（很像 Scikit-Learn 的 StandardScaler），而不是让诸如 mean 和 stds 之类的全局变量悬空：

In [ ]:
class Standardization(keras.layers.Layer):
    def adapt(self, data_sample):
        self.means_ = np.mean(data_sample, axis=0, keepdims=True)
        self.stds_ = np.std(data_sample, axis=0, keepdims=True)
    def call(self, inputs):
        return (inputs - self.means_) / (self.stds_ + keras.backend.epsilon())

在您可以使用此标准化层之前，您需要通过调用 adapt() 方法并将数据样本传递给它来使其适应您的数据集。 这将允许它为每个特征使用适当的均值和标准差：

In [ ]:
std_layer = Standardization()
std_layer.adapt(data_sample)

该样本必须足够大才能代表您的数据集，但它不必是完整的训练集：通常，随机选择几百个实例就足够了（但是，这取决于您的任务）。 接下来，您可以像使用普通层一样使用此预处理层：

In [ ]:
model = keras.Sequential()
model.add(std_layer) [...] # create the rest of the model
model.compile([...])
model.fit([...])

如果您认为 Keras 应该包含这样的标准化层，那么这里有一些好消息给您：当您阅读本文时，keras.layers.Normalization 层可能已经可用。 它的工作方式与我们自定义的标准化层非常相似：首先，创建层，然后通过将数据样本传递给 adapt() 方法使其适应您的数据集，最后正常使用该层。

现在让我们看看分类特征。 我们将首先将它们编码为 one-hot 向量。

### 使用 One-Hot 向量编码分类特征

考虑我们在第 2 章中探索的加利福尼亚住房数据集中的 Ocean_proximity 特征：它是一个具有五个可能值的分类特征：“<1H OCEAN”、“INLAND”、“NEAR OCEAN”、“NEAR BAY”和“ISLAND”。 在将其提供给神经网络之前，我们需要对该特征进行编码。 由于类别很少，我们可以使用one-hot编码。 为此，我们首先需要将每个类别映射到它的索引（0 到 4），这可以使用查找表来完成：

In [46]:
vocab = ["<1H OCEAN", "INLAND", "NEAR OCEAN", "NEAR BAY", "ISLAND"]
indices = tf.range(len(vocab), dtype=tf.int64)

time: 7.11 ms (started: 2021-08-18 02:46:47 +08:00)


In [47]:
table_init = tf.lookup.KeyValueTensorInitializer(vocab, indices)
num_oov_buckets = 2
table = tf.lookup.StaticVocabularyTable(table_init, num_oov_buckets)

time: 3.32 ms (started: 2021-08-18 02:47:00 +08:00)


让我们来看看这段代码：
* 我们首先定义词汇表：这是所有可能类别的列表。
* 然后我们创建一个具有相应索引（0 到 4）的张量。
* 接下来，我们为查找表创建一个初始化程序，将类别列表及其相应的索引传递给它。 在这个例子中，我们已经有了这些数据，所以我们使用了一个 KeyValueTensorInitializer； 但是如果类别在文本文件中列出（每行一个类别），我们将使用 TextFileInitializer 代替。
* 在最后两行中，我们创建了查找表，为其提供初始值设定项并指定词表外 (oov) 桶的数量。 如果我们查找词汇表中不存在的类别，查找表将计算该类别的散列并使用它来将未知类别分配给 oov 存储桶之一。 它们的索引在已知类别之后开始，因此在此示例中，两个 oov 存储桶的索引是 5 和 6。

为什么要使用 oov 存储桶？ 那么，如果类别的数量很大（例如，邮政编码、城市、单词、产品或用户）并且数据集也很大，或者它不断变化，那么获取类别的完整列表可能并不方便。 一种解决方案是根据数据样本（而不是整个训练集）定义词汇表，并为不在数据样本中的其他类别添加一些 oov 桶。 您希望在训练期间找到的未知类别越多，您应该使用的 oov 存储桶就越多。 确实，如果没有足够的 oov 桶，就会发生冲突：不同的类别最终会出现在同一个桶中，因此神经网络将无法区分它们（至少不是基于这个特征）。

现在让我们使用查找表将一小批分类特征编码为一个热向量：

In [48]:
category = tf.constant(["NEAR BAY", "DESERT", "内陆", "内陆"])

time: 893 µs (started: 2021-08-18 02:49:34 +08:00)


In [50]:
cat_indices = table.lookup(category)

time: 61.6 ms (started: 2021-08-18 02:49:51 +08:00)


In [51]:
 cat_indices

<tf.Tensor: shape=(4,), dtype=int64, numpy=array([3, 5, 5, 5])>

time: 4.33 ms (started: 2021-08-18 02:49:57 +08:00)


In [52]:
 cat_one_hot = tf.one_hot(cat_indices, depth=len(vocab) + num_oov_buckets)

time: 4.27 ms (started: 2021-08-18 02:50:05 +08:00)


In [53]:
 cat_one_hot

<tf.Tensor: shape=(4, 7), dtype=float32, numpy=
array([[0., 0., 0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.],
       [0., 0., 0., 0., 0., 1., 0.]], dtype=float32)>

time: 4.26 ms (started: 2021-08-18 02:50:10 +08:00)


根据经验，如果类别数低于 10，则通常采用 one-hot 编码（但您的里程可能会有所不同！）。 如果类别数大于 50（使用哈希桶时经常出现这种情况），则嵌入通常更可取。 在 10 到 50 个类别中，您可能希望对这两个选项进行试验，看看哪一个最适合您的用例。

### 使用嵌入编码分类特征

嵌入是表示类别的可训练密集向量。默认情况下，嵌入是随机初始化的，因此例如“NEAR BAY”类别最初可以由一个随机向量表示，例如 [0.131, 0.890]，而“NEAR OCEAN”类别可能由另一个随机向量表示，例如 [ 0.631, 0.791]。在此示例中，我们使用 2D 嵌入，但维度数是您可以调整的超参数。由于这些嵌入是可训练的，它们会在训练过程中逐渐改进；并且由于它们代表非常相似的类别，因此梯度下降肯定会最终将它们推得更近，同时它会倾向于将它们从“内陆”类别的嵌入中移开（见图 13-4）。事实上，表示越好，神经网络就越容易做出准确的预测，因此训练倾向于使嵌入成为类别的有用表示。这称为表示学习（我们将在第 17 章中看到其他类型的表示学习）。

![](https://tva1.sinaimg.cn/large/008i3skNgy1gtkd9l785nj618o0l476e02.jpg)

Keras 提供了一个 keras.layers.Embedding 层来处理嵌入矩阵（默认情况下可训练）； 当层被创建时，它随机初始化嵌入矩阵，然后当它被一些类别索引调用时，它返回嵌入矩阵中这些索引处的行：

In [ ]:
embedding = keras.layers.Embedding(input_dim=len(vocab) + num_oov_buckets,
                                   output_dim=embedding_dim)

In [ ]:
embedding(cat_indices)

将所有内容放在一起，我们现在可以创建一个 Keras 模型，该模型可以处理分类特征（以及常规数值特征）并学习每个类别（以及每个 oov 存储桶）的嵌入：

In [ ]:
regular_inputs = keras.layers.Input(shape=[8])
categories = keras.layers.Input(shape=[], dtype=tf.string)
cat_indices = keras.layers.Lambda(lambda cats: table.lookup(cats))(categories)
cat_embed = keras.layers.Embedding(input_dim=6, output_dim=2)(cat_indices)
encoded_inputs = keras.layers.concatenate([regular_inputs, cat_embed])
outputs = keras.layers.Dense(1)(encoded_inputs)
model = keras.models.Model(inputs=[regular_inputs, categories],
outputs=[outputs])

该模型需要两个输入：一个包含每个实例八个数字特征的常规输入，以及一个分类输入（每个实例包含一个分类特征）。 它使用 Lambda 层查找每个类别的索引，然后查找这些索引的嵌入。 接下来，它将嵌入和常规输入连接起来，以提供编码输入，这些输入准备好馈送到神经网络。 此时我们可以添加任何类型的神经网络，但我们只需添加一个密集输出层，然后我们创建 Keras 模型。
当 keras.layers.TextVectorization 层可用时，您可以调用它的 adapt() 方法使其从数据样本中提取词汇表（它将负责为您创建查找表）。 然后您可以将其添加到您的模型中，它将执行索引查找（替换前面代码示例中的 Lambda 层）。

> One-hot 编码后跟一个 Dense 层（没有激活函数和偏差）相当于一个嵌入层。 然而，嵌入层使用更少的计算（当嵌入矩阵的大小增加时，性能差异变得明显）。 Dense 层的权重矩阵起到了嵌入矩阵的作用。 例如，使用大小为 20 的 one-hot 向量和具有 10 个单位的 Dense 层等效于使用 input_dim=20 和 output_dim=10 的嵌入层。 因此，使用比 Embedding 层之后的层中的单元数更多的嵌入维度将是一种浪费。

现在让我们更仔细地看看 Keras 预处理层。

### Keras 预处理层

TensorFlow 团队正在致力于提供一组标准的 Keras 预处理层。 当您阅读本文时，它们可能已经可用； 但是，届时 API 可能会略有变化，因此如果出现任何异常行为，请参阅本章的笔记本。 这个新的 API 可能会取代现有的 Feature Columns API，后者更难使用且不太直观（如果你想了解更多关于 Feature Columns API 的信息，请查看本章的笔记本）。

我们已经讨论了其中的两个层：将执行特征标准化的 keras.layers.Normalization 层（它相当于我们之前定义的 Standardization 层），以及能够将输入中的每个单词编码为 它在词汇表中的索引。 在这两种情况下，您都可以创建层，使用数据样本调用它的 adapt() 方法，然后在模型中正常使用该层。 其他预处理层将遵循相同的模式。

该 API 还将包括一个 keras.layers.Discretization 层，它将连续数据切割成不同的 bin 并将每个 bin 编码为一个单热向量。 例如，您可以使用它将价格离散为三类（低、中、高），它们将被编码为 [1, 0, 0]、[0, 1, 0] 和 [0, 0, 1] ]， 分别。 当然，这会丢失很多信息，但在某些情况下，它可以帮助模型检测在仅查看连续值时不明显的模式。

> Discretization 层是不可微的，它应该只在模型开始时使用。 实际上，模型的预处理层在训练过程中会被冻结，因此它们的参数不会受到梯度下降的影响，因此它们不需要可微。 这也意味着您不应该直接在自定义预处理层中使用 Embedding 层，如果您希望它可训练：相反，应将其单独添加到您的模型中，如前面的代码示例所示。

还可以使用 PreprocessingStage 类链接多个预处理层。 例如，以下代码将创建一个预处理管道，该管道将首先标准化输入，然后离散化它们（这可能会让您想起 Scikit-Learn 管道）。 将此管道调整为数据样本后，您可以像模型中的常规层一样使用它（但同样，仅在模型的开头，因为它包含不可微分的预处理层）：

In [ ]:
normalization = keras.layers.Normalization()
discretization = keras.layers.Discretization([...])
pipeline = keras.layers.PreprocessingStage([normalization, discretization])
pipeline.adapt(data_sample)

TextVectorization 层还可以选择输出字数向量而不是字索引。例如，如果词汇表包含三个单词，比如 ["and", "basketball", "more"]，那么文本 "more and more" 将映射到向量 [1, 0, 2]：单词 " and”出现一次，“basketball”这个词根本没有出现，“more”这个词出现了两次。这种文本表示被称为词袋，因为它完全失去了词的顺序。像“and”这样的常用词在大多数文本中都有很大的价值，即使它们通常是最不有趣的（例如，在文本“越来越多的篮球”中，“篮球”这个词显然是最重要的，正是因为它是不是很常用的词）。因此，应以降低频繁词重要性的方式对字数进行归一化。一种常见的方法是将每个单词计数除以该单词出现的训练实例总数的对数。这种技术称为术语频率×逆文档频率（TF-IDF）。例如，假设单词“and”、“basketball”和“more”分别出现在训练集中的 200、10 和 100 个文本实例中：在这种情况下，最终向量将为 [1/log( 200), 0/log(10), 2/log(100)]，大约等于 [0.19, 0., 0.43]。 TextVectorization 层将（可能）有一个选项来执行 TF-IDF。

> 如果标准预处理层不足以完成您的任务，您仍然可以选择创建自己的自定义预处理层，就像我们之前对 Standardization 类所做的一样。 使用 adapt() 方法创建 keras.layers.PreprocessingLayer 类的子类，该方法应采用 data_sample 参数和可选的额外 reset_state 参数：如果为 True，则 adapt() 方法应在计算新状态之前重置任何现有状态 ; 如果为 False，它应该尝试更新现有状态。

如您所见，这些 Keras 预处理层将使预处理变得更加容易！ 现在，无论您选择编写自己的预处理层还是使用 Keras（甚至使用 Feature Columns API），所有预处理都将即时完成。 然而，在训练期间，最好提前执行预处理。 让我们看看我们为什么要这样做以及我们将如何去做。

In [56]:
dir(tf.keras.preprocessing)

['__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'image',
 'image_dataset_from_directory',
 'sequence',
 'text',
 'text_dataset_from_directory',
 'timeseries_dataset_from_array']

time: 3.39 ms (started: 2021-08-18 03:06:30 +08:00)


In [57]:
dir(tf.keras.preprocessing.image)

['DirectoryIterator',
 'ImageDataGenerator',
 'Iterator',
 'NumpyArrayIterator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'apply_affine_transform',
 'apply_brightness_shift',
 'apply_channel_shift',
 'array_to_img',
 'img_to_array',
 'load_img',
 'random_brightness',
 'random_channel_shift',
 'random_rotation',
 'random_shear',
 'random_shift',
 'random_zoom',
 'save_img',
 'smart_resize']

time: 3.64 ms (started: 2021-08-18 03:06:47 +08:00)


In [58]:
dir(tf.keras.preprocessing.text)

['Tokenizer',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'hashing_trick',
 'one_hot',
 'text_to_word_sequence',
 'tokenizer_from_json']

time: 3.74 ms (started: 2021-08-18 03:07:01 +08:00)


In [59]:
dir(tf.keras.preprocessing.sequence)

['TimeseriesGenerator',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 '_sys',
 'make_sampling_table',
 'pad_sequences',
 'skipgrams']

time: 3.18 ms (started: 2021-08-18 03:07:14 +08:00)


## TF变换

如果预处理在计算上是昂贵的，那么在训练之前处理它而不是在运行中处理它可能会给你一个显着的加速：在训练前每个实例只对数据进行一次预处理，而不是在训练期间每个实例和每个时代一次。 如前所述，如果数据集小到可以放入 RAM，您可以使用它的 cache() 方法。 但如果它太大，那么像 Apache Beam 或 Spark 这样的工具会有所帮助。 它们让您可以在大量数据上运行高效的数据处理管道，甚至分布在多个服务器上，因此您可以在训练前使用它们对所有训练数据进行预处理。

这很有效，确实可以加快训练速度，但有一个问题：一旦您的模型经过训练，假设您想将其部署到移动应用程序。 在这种情况下，您需要在您的应用程序中编写一些代码，以在将数据提供给模型之前对其进行预处理。 假设您还想将模型部署到 TensorFlow.js 以便它在 Web 浏览器中运行？ 同样，您将需要编写一些预处理代码。 这可能成为维护的噩梦：每当您想要更改预处理逻辑时，您都需要更新您的 Apache Beam 代码、您的移动应用程序代码和您的 JavaScript 代码。 这不仅耗时，而且容易出错：您最终可能会发现训练前执行的预处理操作与在您的应用程序或浏览器中执行的预处理操作之间存在细微差别。 这种训练/服务偏差会导致错误或性能下降。

一种改进是采用经过训练的模型（根据由 Apache Beam 或 Spark 代码预处理的数据进行训练），并在将其部署到您的应用程序或浏览器之前，添加额外的预处理层以进行动态预处理。这肯定更好，因为现在你只有两个版本的预处理代码：Apache Beam 或 Spark 代码，以及预处理层的代码。

但是，如果您可以只定义一次预处理操作呢？这就是 TF Transform 的设计目的。它是 TensorFlow Extended (TFX) 的一部分，这是一个用于生产 TensorFlow 模型的端到端平台。首先，要使用TF Transform等TFX组件，必须安装；它不与 TensorFlow 捆绑在一起。然后，您只需定义一次预处理函数（在 Python 中），使用 TF 变换函数进行缩放、分桶等。你也可以使用任何你需要的 TensorFlow 操作。如果我们只有两个特征，这个预处理函数可能看起来像这样：

In [ ]:
import tensorflow_transform as tft

def preprocess(inputs): # inputs = a batch of input features
    median_age = inputs["housing_median_age"]
    ocean_proximity = inputs["ocean_proximity"]
    standardized_age = tft.scale_to_z_score(median_age)
    ocean_proximity_id = tft.compute_and_apply_vocabulary(ocean_proximity)
    return {
    "standardized_median_age": standardized_age,
    "ocean_proximity_id": ocean_proximity_id
}

接下来，TF Transform 允许您使用 Apache Beam 将此 preprocess() 函数应用到整个训练集（它提供了一个 AnalyzeAndTransformDataset 类，您可以在 Apache Beam 管道中用于此目的）。 在此过程中，它还将计算整个训练集的所有必要统计信息：在本例中，是 Housing_median_age 特征的均值和标准差，以及 Ocean_proximity 特征的词汇表。 计算这些统计数据的组件称为分析器。

## TensorFlow 数据集 (TFDS) 项目

TensorFlow Datasets 项目使下载常见数据集变得非常容易，从 MNIST 或 Fashion MNIST 等小型数据集到 ImageNet 等大型数据集（您将需要相当多的磁盘空间！）。 该列表包括图像数据集、文本数据集（包括翻译数据集）和音频和视频数据集。 您可以访问 https://homl.info/tfds 查看完整列表以及每个数据集的描述。

TFDS 未与 TensorFlow 捆绑，因此您需要安装 tensorflow 数据集库（例如，使用 pip）。 然后调用 tfds.load() 函数，它会下载你想要的数据（除非它之前已经下载过）并将数据作为数据集字典返回（通常一个用于训练，一个用于测试，但这取决于 您选择的数据集）。 例如，让我们下载 MNIST：

In [62]:
import tensorflow_datasets as tfds

dataset = tfds.load(name="mnist")
mnist_train, mnist_test = dataset["train"], dataset["test"]

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-train.tfrecord...:   0%|          | 0/60000 [00:00<?, ? examples/s]

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling mnist-test.tfrecord...:   0%|          | 0/10000 [00:00<?, ? examples/s]

Dataset mnist downloaded and prepared to /home/huangwei/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.
time: 1min 46s (started: 2021-08-18 03:15:24 +08:00)


然后，您可以应用您想要的任何转换（通常是混洗、批处理和预取），并准备好训练您的模型。 这是一个简单的例子：

In [ ]:
mnist_train = mnist_train.shuffle(10000).batch(32).prefetch(1)
for item in mnist_train:
images = item["image"]
labels = item["label"] [...]

load() 函数会对其下载的每个数据分片进行混洗（仅适用于训练集）。 这可能还不够，所以最好再打乱训练数据。

请注意，数据集中的每个项目都是一个包含特征和标签的字典。 但是 Keras 期望每个项目都是一个包含两个元素（同样是特征和标签）的元组。 您可以使用 map() 方法转换数据集，如下所示：

In [63]:
mnist_train = mnist_train.shuffle(10000).batch(32)
mnist_train = mnist_train.map(lambda items: (items["image"], items["label"]))
mnist_train = mnist_train.prefetch(1)

time: 37.2 ms (started: 2021-08-18 03:17:54 +08:00)


但是通过设置 as_supervised=True 来让 load() 函数为你做这件事更简单（显然这仅适用于标记数据集）。 如果需要，您还可以指定批量大小。 然后您可以将数据集直接传递给您的 tf.keras 模型：

In [ ]:
dataset = tfds.load(name="mnist", batch_size=32, as_supervised=True)
mnist_train = dataset["train"].prefetch(1)
model = keras.models.Sequential([...])
model.compile(loss="sparse_categorical_crossentropy", optimizer="sgd")
model.fit(mnist_train, epochs=5)

这是一个相当技术性的章节，你可能会觉得它离神经网络的抽象美有点远，但事实是深度学习通常涉及大量数据，并且知道如何有效地加载、解析和预处理它 是一项至关重要的技能。 在下一章中，我们将研究卷积神经网络，它是用于图像处理和许多其他应用的最成功的神经网络架构之一。